In [2]:
from PIL import Image
import numpy as np

In [3]:
def crop(ImgArr):
    x_disp = 0; y_disp = 0; x2_disp = 0;
    
    i = 0
    while sum(sum(ImgArr[i])) == 0:
        ImgArr = np.delete(ImgArr, i, 0)
        y_disp += 1
    i = -1
    while sum(sum(ImgArr[i])) == 0:
        ImgArr = np.delete(ImgArr, i, 0)
        #y_disp += 1
    i = 0
    while sum(sum(ImgArr[:,i])) == 0:
        ImgArr = np.delete(ImgArr, i, 1)
        x_disp += 1
    i = -1
    while sum(sum(ImgArr[:,i])) == 0:
        ImgArr = np.delete(ImgArr, i, 1)
        x2_disp += 1
    return ImgArr, x_disp, y_disp, x2_disp

In [4]:
ALPHA = "ALPHA"
alpha_bit = 0xFE

def RGBA_to_bitmap(col):
    #8 BIT: BBBGGGRR
    if len(col) == 4:
        if col[3] == 0: return ALPHA
    col = np.array(col)/255.
    bitcol = int(round(col[0]*3) + round(col[1]*7)*4 + round(col[2]*7)*4*8)
    if bitcol == alpha_bit: bitcol += 1
    return bitcol

def bitmap_to_RGBA(bitcol):
    #8 BIT: BBBGGGRR
    if bitcol == ALPHA: return [0,0,0,0]
    R = bitcol%4
    G = bitcol//4 % 8
    B = bitcol//(4*8)
    return [int(x) for x in [R/3.*255, G/7.*255, B/7.*255, 255]]
    
def GetBitmap(ImgArr):
    shape = ImgArr.shape
    Bitmap = [[RGBA_to_bitmap(COL) for COL in row] for row in ImgArr]
    return {"X": shape[1], "Y": shape[0]}, Bitmap

def Bitmap_to_string(Bitmap):
    return ", ".join([str(item) for sublist in Bitmap for item in sublist])

def Bitmap_to_image(Bitmap):
    return Image.fromarray(np.array([[bitmap_to_RGBA(COL) for COL in row] for row in Bitmap], dtype="uint8"))

In [7]:
def Filename_to_bitmap(FILENAME, RESOLUTION_x, Orientation = "xy"):
    '''
    FILENAME must be of an image with an alpha channel
    RESOLUTION_x is an estimate of the x-resolution of the final bitmap
    
    returns:
        STRING, string of final bitmap. Just copy and paste it
        shape: shape of final bitmap (after cropping extra alpha rows and columns)
    '''
    
    Img = Image.open(FILENAME)
    RX = np.asarray(Img).shape[0]
    RY = np.asarray(Img).shape[1]
    if Orientation == "yx": t = RX; RX = RY; RY = t;
    scale = RESOLUTION_x/RX
    Img = Img.resize((round(RX*scale), round(RY*scale)), Image.NEAREST)
    ImgArr = np.asarray(Img)
    ImgArrCrop, x_disp, y_disp, _ = crop(ImgArr)
    shape, Bitmap = GetBitmap(ImgArrCrop)
    Bitmap = [[col if col != 0 else 0xFF for col in row] for row in Bitmap]
    STRING = Bitmap_to_string(Bitmap)
    preview = Bitmap_to_image(Bitmap)
    
    return STRING, shape, preview, x_disp, y_disp

In [11]:
#FILENAME = r"C:\Users\nailu\Downloads\eheh.png"
#FILENAME = r"C:\Users\nailu\Downloads\grass.png"
#FILENAME = r"C:\Users\nailu\Downloads\flag (3).jpg"
#FILENAME = r"C:\Users\nailu\Downloads\hamburger_menu.png"
FILENAME = r"info_sprite.png"
STRING, shape, preview, x_disp, y_disp = Filename_to_bitmap(FILENAME, 10, Orientation = "yx")

In [12]:
shape

{'X': 8, 'Y': 9}

In [13]:
preview

In [14]:
STRING

'ALPHA, ALPHA, 209, 209, 209, 209, ALPHA, ALPHA, ALPHA, 209, 209, 209, 209, 209, 209, ALPHA, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 255, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 176, 209, 209, 255, 255, 209, 209, 176, ALPHA, 176, 209, 209, 209, 209, 176, ALPHA, ALPHA, ALPHA, ALPHA, 176, 176, ALPHA, ALPHA, ALPHA'

In [15]:
from PIL import GifImagePlugin

In [142]:
def replace_alpha(col, frame):
    frame = frame.copy();
    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            if (sum([1 for X,Y in zip(frame[i][j], col) if X != Y]) == 0):
                for k in range(4): frame[i][j][k] = 0
    return frame

In [154]:
FILENAME = r"./cutie2.gif"
SPRITENAME = "RUN"
RESOLUTION_X = 45 

GIF = Image.open(FILENAME)
n_frames = GIF.n_frames
FRAMES = []
for i in range(n_frames):
    GIF.seek(i)
    img = GIF.copy()
    FRAMES.append(img)
    
scale = float(RESOLUTION_X)/np.asarray(FRAMES[0]).shape[1]
FRAMES_resized = [Img.resize((int(np.asarray(Img).shape[1] * scale), int(np.asarray(Img).shape[0] * scale)), Image.NEAREST) for Img in FRAMES]
FRAMES_RGBA = [np.asarray(Img.convert("RGBA")) for Img in FRAMES_resized]
FRAMES_RGBA = [replace_alpha([192, 192, 192, 255], frame) for frame in FRAMES_RGBA]
FRAMES_cropped = [crop(Img)[0] for Img in FRAMES_RGBA]
FRAMES_disp = [(crop(Img)[1], crop(Img)[2], crop(Img)[3]) for Img in FRAMES_RGBA]
FRAMES_disp_2 = [disp[2] - FRAMES_disp[0][0] for disp in FRAMES_disp]
FRAMES_disp = [(disp[0] - FRAMES_disp[0][0], disp[1] - FRAMES_disp[0][1]) for disp in FRAMES_disp]
Bitmap = [GetBitmap(Img) for Img in FRAMES_cropped]
STRING = [(shape, Bitmap_to_string(bitmap)) for shape, bitmap in Bitmap]

print(Bitmap[0][0])

DISP_STR = "{" + ", ".join(["{0}, {1}".format(disp[0], disp[1]) for disp in FRAMES_disp]) + "};"
DISP2_STR = "{" + ", ".join([str(n) for n in FRAMES_disp_2]) + "};"
CODE = '''const char {0}_disp[{1}] PROGMEM = {2}\n'''.format(SPRITENAME, len(STRING)*2, DISP_STR)
CODE += "const char {0}_disp2[{1}] PROGMEM = {2}\n".format(SPRITENAME, len(STRING), DISP2_STR)
CODE += '''const unsigned char {0}_XY[{1}] PROGMEM = '''.format(SPRITENAME, len(STRING)*2) + "{" + ", ".join(["{0}, {1}".format(shape['X'],shape['Y']) for shape, _ in Bitmap]) + "};" + '''
''' + "\n".join(["const unsigned char {2}_{0}[{3}] PROGMEM = {1} ".format(i, "{" + string[1] + "};", SPRITENAME, Bitmap[i][0]['X']*Bitmap[i][0]['Y']) for i, string in enumerate(STRING)]) + '''

const long {0}_ARR[{1}] PROGMEM = '''.format(SPRITENAME, len(STRING)) + "{" + ", ".join(["{0}_{1}".format(SPRITENAME, i) for i in range(len(STRING))]) + "};"

with open("{}_code.txt".format(SPRITENAME), "w") as f: f.write(CODE)

{'X': 15, 'Y': 19}


In [180]:
import os

FOLDER = r".\kaho_idle"
SPRITENAME = "IDLE"
RESOLUTION_X = 29

#FOLDER = r".\tree_sprites"
#SPRITENAME = "TREE"
#RESOLUTION_X = 30

Img_Name_List = os.listdir(FOLDER)
FRAMES = [Image.open(FOLDER + "\\" + img) for img in Img_Name_List]

#FRAMES = [img.crop((50, 0, 130, 140)) for img in FRAMES]
    
scale = float(RESOLUTION_X)/np.asarray(FRAMES[0]).shape[1]
FRAMES_resized = [Img.resize((int(np.asarray(Img).shape[1] * scale), int(np.asarray(Img).shape[0] * scale)), Image.NEAREST) for Img in FRAMES]
FRAMES_RGBA = [np.asarray(Img.convert("RGBA")) for Img in FRAMES_resized]
FRAMES_cropped = [crop(Img)[0] for Img in FRAMES_RGBA]
FRAMES_disp = [(crop(Img)[1], crop(Img)[2], crop(Img)[3]) for Img in FRAMES_RGBA]
FRAMES_disp_2 = [disp[2] - FRAMES_disp[0][0] for disp in FRAMES_disp]
FRAMES_disp = [(disp[0] - FRAMES_disp[0][0], disp[1] - FRAMES_disp[0][1]) for disp in FRAMES_disp]
Bitmap = [GetBitmap(Img) for Img in FRAMES_cropped]
STRING = [(shape, Bitmap_to_string(bitmap)) for shape, bitmap in Bitmap]

print(Bitmap[-1][0])
print(len(Bitmap))
print(sum([b[0]["X"]*b[0]["Y"] for b in Bitmap]))

DISP_STR = "{" + ", ".join(["{0}, {1}".format(disp[0], disp[1]) for disp in FRAMES_disp]) + "};"
DISP2_STR = "{" + ", ".join([str(n) for n in FRAMES_disp_2]) + "};"
CODE = '''const char {0}_disp[{1}] PROGMEM = {2}\n'''.format(SPRITENAME, len(STRING)*2, DISP_STR)
CODE += "const char {0}_disp2[{1}] PROGMEM = {2}\n".format(SPRITENAME, len(STRING), DISP2_STR)
CODE += '''const unsigned char {0}_XY[{1}] PROGMEM = '''.format(SPRITENAME, len(STRING)*2) + "{" + ", ".join(["{0}, {1}".format(shape['X'],shape['Y']) for shape, _ in Bitmap]) + "};" + '''
''' + "\n".join(["const unsigned char {2}_{0}[{3}] PROGMEM = {1} ".format(i, "{" + string[1] + "};", SPRITENAME, Bitmap[i][0]['X']*Bitmap[i][0]['Y']) for i, string in enumerate(STRING)]) + '''

const long {0}_ARR[{1}] PROGMEM = '''.format(SPRITENAME, len(STRING)) + "{" + ", ".join(["{0}_{1}".format(SPRITENAME, i) for i in range(len(STRING))]) + "};"

with open("{}_code.txt".format(SPRITENAME), "w") as f: f.write(CODE)

{'X': 11, 'Y': 22}
6
1562


In [30]:
from functools import reduce 
reduce(lambda a,b: a*b, [2,3,4])

24

In [190]:
Bitmap_to_image(Bitmap[0][1])

In [ ]:
FRAMES[-2].crop((50, 0, 130, 143))

In [43]:
FRAMES[0].size

(158, 269)

In [124]:
a.shape

(51, 30, 4)

In [169]:
import os

FOLDER = r".\tree_sprites"
SPRITENAME = "TREE"
RESOLUTION_X = 25

Img_Name_List = os.listdir(FOLDER)
FRAMES = [Image.open(FOLDER + "\\" + img) for img in Img_Name_List]

FRAMES = [img.crop((50, 0, 130, 140)) for img in FRAMES]
    
scale = float(RESOLUTION_X)/np.asarray(FRAMES[0]).shape[1]
FRAMES_resized = [Img.resize((int(np.asarray(Img).shape[1] * scale), int(np.asarray(Img).shape[0] * scale)), Image.NEAREST) for Img in FRAMES]
FRAMES_RGBA = [np.asarray(Img.convert("RGBA")) for Img in FRAMES_resized]
FRAMES_cropped = [crop(Img)[0] for Img in FRAMES_RGBA]
FRAMES_disp = [(crop(Img)[1], crop(Img)[2], crop(Img)[3]) for Img in FRAMES_RGBA]
FRAMES_disp_2 = [disp[2] - FRAMES_disp[0][0] for disp in FRAMES_disp]
FRAMES_disp = [(disp[0] - FRAMES_disp[0][0], disp[1] - FRAMES_disp[0][1]) for disp in FRAMES_disp]
Bitmap = [GetBitmap(Img) for Img in FRAMES_cropped]
STRING = [(shape, Bitmap_to_string(bitmap)) for shape, bitmap in Bitmap]

print(Bitmap[-1][0])
print(len(Bitmap))
print(sum([b[0]["X"]*b[0]["Y"] for b in Bitmap]))

DISP_STR = "{" + ", ".join(["{0}, {1}".format(disp[0], disp[1]) for disp in FRAMES_disp]) + "};"
DISP2_STR = "{" + ", ".join([str(n) for n in FRAMES_disp_2]) + "};"
CODE = '''const char {0}_disp[{1}] PROGMEM = {2}\n'''.format(SPRITENAME, len(STRING)*2, DISP_STR)
CODE += "const char {0}_disp2[{1}] PROGMEM = {2}\n".format(SPRITENAME, len(STRING), DISP2_STR)
CODE += '''const unsigned char {0}_XY[{1}] PROGMEM = '''.format(SPRITENAME, len(STRING)*2) + "{" + ", ".join(["{0}, {1}".format(shape['X'],shape['Y']) for shape, _ in Bitmap]) + "};" + '''
''' + "\n".join(["const unsigned char {2}_{0}[{3}] PROGMEM = {1} ".format(i, "{" + string[1] + "};", SPRITENAME, Bitmap[i][0]['X']*Bitmap[i][0]['Y']) for i, string in enumerate(STRING)]) + '''

const long {0}_ARR[{1}] PROGMEM = '''.format(SPRITENAME, len(STRING)) + "{" + ", ".join(["{0}_{1}".format(SPRITENAME, i) for i in range(len(STRING))]) + "};"

with open("{}_code.txt".format(SPRITENAME), "w") as f: f.write(CODE)

{'X': 25, 'Y': 33}
5
1792
